In [1]:
import os
import mlflow
import requests
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from json import dump, load


In [2]:
api_url = "http://10.43.101.149/data?group_number=1"
response = requests.get(api_url)
if response.status_code == 200:
    data = response.json()
    with open("api/data/covertype.json", "w") as outfile:
        dump(data, outfile)
    print("Data fetched from API and saved locally.")
else:
    print("Error fetching data from API.")

Error fetching data from API.


In [3]:
# Load data from JSON into DataFrame
with open('./data/covertype.json') as f:
    data = load(f)
df = pd.DataFrame(data['data'])
# Assign column names because json just contains data without headers (column names)
column_names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
                'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
                'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
                'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type',
                'Cover_Type']
df.columns = column_names

# Set the target values
y = df['Cover_Type']#.values

# Set the input values
df.drop('Cover_Type', axis=1, inplace=True)
X = df#.values

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore'),
                                        ["Wilderness_Area", "Soil_Type"]),
                                      remainder='passthrough') # pass all the numeric values through the pipeline without any changes.

column_trans


ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Wilderness_Area', 'Soil_Type'])])

In [5]:
pipe = Pipeline(steps=[("column_trans", column_trans),("scaler", StandardScaler(with_mean=False)), ("RandomForestClassifier", RandomForestClassifier())])

pipe

Pipeline(steps=[('column_trans',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Wilderness_Area',
                                                   'Soil_Type'])])),
                ('scaler', StandardScaler(with_mean=False)),
                ('RandomForestClassifier', RandomForestClassifier())])

In [6]:
param_grid =  {'RandomForestClassifier__max_depth': [1,2,3,10], 'RandomForestClassifier__n_estimators': [10,11]}

search = GridSearchCV(pipe, param_grid, n_jobs=2)
search


GridSearchCV(estimator=Pipeline(steps=[('column_trans',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Wilderness_Area',
                                                                          'Soil_Type'])])),
                                       ('scaler',
                                        StandardScaler(with_mean=False)),
                                       ('RandomForestClassifier',
                                        RandomForestClassifier())]),
             n_jobs=2,
             param_grid={'RandomForestClassifier__max_depth': [1, 2, 3, 10],
                         'RandomForestClassifier__n_estimators': [10, 11]})

In [7]:
search.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__column_trans', 'estimator__scaler', 'estimator__RandomForestClassifier', 'estimator__column_trans__n_jobs', 'estimator__column_trans__remainder', 'estimator__column_trans__sparse_threshold', 'estimator__column_trans__transformer_weights', 'estimator__column_trans__transformers', 'estimator__column_trans__verbose', 'estimator__column_trans__verbose_feature_names_out', 'estimator__column_trans__onehotencoder', 'estimator__column_trans__onehotencoder__categories', 'estimator__column_trans__onehotencoder__drop', 'estimator__column_trans__onehotencoder__dtype', 'estimator__column_trans__onehotencoder__handle_unknown', 'estimator__column_trans__onehotencoder__max_categories', 'estimator__column_trans__onehotencoder__min_frequency', 'estimator__column_trans__onehotencoder__sparse', 'estimator__column_trans__onehotencoder__sparse_output', 'estimator__scaler__copy', 'estimator__scaler__wit

In [8]:
import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://0.0.0.0:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

# connect to mlflow
mlflow.set_tracking_uri("http://0.0.0.0:5000")
mlflow.set_experiment("mlflow_tracking_examples")

mlflow.autolog(log_model_signatures=True, log_input_examples=True)

with mlflow.start_run(run_name="autolog_with_pipeline") as run:
    search.fit(X_train, y_train)

MlflowException: API request to http://0.0.0.0:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='0.0.0.0', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=mlflow_tracking_examples (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fbf08ad61f0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
print('tracking uri:', mlflow.get_tracking_uri())
print('artifact uri:', mlflow.get_artifact_uri())

In [ ]:
import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://localhost:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

# connect to mlflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("mlflow_tracking_examples")

mlflow.sklearn.autolog(log_model_signatures=True, log_input_examples=True, registered_model_name="modelo1")

with mlflow.start_run(run_name="autolog_pipe_model_reg") as run:
    search.fit(X_train, y_train)

In [ ]:
import mlflow

import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://localhost:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

# connect to mlflow
mlflow.set_tracking_uri("http://localhost:5000")

model_name = "modelo1"

# logged_model = 'runs:/71428bebed2b4feb9635714ea3cdb562/model'
model_production_uri = "models:/{model_name}/production".format(model_name=model_name)

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri=model_production_uri)
loaded_model
example_test = X_test.iloc[0].to_frame().T
#print(example_test)
print('real: ', y_test.iloc[0])
print('prediction: ', loaded_model.predict(example_test))

In [ ]:
X_test.iloc[0].to_frame().T